
<br>
============================================<br>
Estimate anisotropy in a 3D microscopy image<br>
============================================<br>
In this tutorial, we compute the structure tensor of a 3D image.<br>
For a general introduction to 3D image processing, please refer to<br>
:ref:`sphx_glr_auto_examples_applications_plot_3d_image_processing.py`.<br>
The data we use here are sampled from an image of kidney tissue obtained by<br>
confocal fluorescence microscopy (more details at [1]_ under<br>
``kidney-tissue-fluorescence.tif``).<br>
.. [1] https://gitlab.com/scikit-image/data/#data<br>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import plotly.express as px
from skimage import (
    data, feature
)

###################################################################<br>
Load image<br>
==========<br>
This biomedical image is available through `scikit-image`'s data registry.

In [ ]:
data = data.kidney()

###################################################################<br>
What exactly are the shape and size of our 3D multichannel image?

In [ ]:
print(f'number of dimensions: {data.ndim}')
print(f'shape: {data.shape}')
print(f'dtype: {data.dtype}')

###################################################################<br>
For the purposes of this tutorial, we shall consider only the second color<br>
channel, which leaves us with a 3D single-channel image. What is the range<br>
of values?

In [ ]:
n_plane, n_row, n_col, n_chan = data.shape
v_min, v_max = data[:, :, :, 1].min(), data[:, :, :, 1].max()
print(f'range: ({v_min}, {v_max})')

###################################################################<br>
Let us visualize the middle slice of our 3D image.

In [ ]:
px.imshow(
    data[n_plane // 2, :, :, 1],
    zmin=v_min,
    zmax=v_max,
    labels={'x': 'col', 'y': 'row', 'color': 'intensity'}
)

###################################################################<br>
Let us pick a specific region, which shows relative X-Y isotropy. In<br>
contrast, the gradient is quite different (and, for that matter, weak) along<br>
Z.

In [ ]:
sample = data[5:13, 380:410, 370:400, 1]
step = 3
cols = sample.shape[0] // step + 1
_, axes = plt.subplots(nrows=1, ncols=cols, figsize=(16, 8))

In [ ]:
for it, (ax, image) in enumerate(zip(axes.flatten(), sample[::step])):
    ax.imshow(image, cmap='gray', vmin=v_min, vmax=v_max)
    ax.set_title(f'Plane = {5 + it * step}')
    ax.set_xticks([])
    ax.set_yticks([])

###################################################################<br>
To view the sample data in 3D, run the following code:<br>
<br>
.. code-block:: python<br>
<br>
    import plotly.graph_objects as go<br>
<br>
    (n_Z, n_Y, n_X) = sample.shape<br>
    Z, Y, X = np.mgrid[:n_Z, :n_Y, :n_X]<br>
<br>
    fig = go.Figure(<br>
        data=go.Volume(<br>
            x=X.flatten(),<br>
            y=Y.flatten(),<br>
            z=Z.flatten(),<br>
            value=sample.flatten(),<br>
            opacity=0.5,<br>
            slices_z=dict(show=True, locations=[4])<br>
        )<br>
    )<br>
    fig.show()

###################################################################<br>
Compute structure tensor<br>
========================<br>
Let us visualize the bottom slice of our sample data and determine the<br>
typical size for strong variations. We shall use this size as the<br>
'width' of the window function.

In [ ]:
px.imshow(
    sample[0, :, :],
    zmin=v_min,
    zmax=v_max,
    labels={'x': 'col', 'y': 'row', 'color': 'intensity'},
    title='Interactive view of bottom slice of sample data.'
)

###################################################################<br>
About the brightest region (i.e., at row ~ 22 and column ~ 17), we can see<br>
variations (and, hence, strong gradients) over 2 or 3 (resp. 1 or 2) pixels<br>
across columns (resp. rows). We may thus choose, say, ``sigma = 1.5`` for<br>
the window<br>
function. Alternatively, we can pass sigma on a per-axis basis, e.g.,<br>
``sigma = (1, 2, 3)``. Note that size 1 sounds reasonable along the first<br>
(Z, plane) axis, since the latter is of size 8 (13 - 5). Viewing slices in<br>
the X-Z or Y-Z planes confirms it is reasonable.

In [ ]:
sigma = (1, 1.5, 2.5)
A_elems = feature.structure_tensor(sample, sigma=sigma)

###################################################################<br>
We can then compute the eigenvalues of the structure tensor.

In [ ]:
eigen = feature.structure_tensor_eigenvalues(A_elems)
eigen.shape

###################################################################<br>
Where is the largest eigenvalue?

In [ ]:
coords = np.unravel_index(eigen.argmax(), eigen.shape)
assert coords[0] == 0  # by definition
coords

###################################################################<br>
.. note::<br>
   The reader may check how robust this result (coordinates<br>
   ``(plane, row, column) = coords[1:]``) is to varying ``sigma``.<br>
<br>
Let us view the spatial distribution of the eigenvalues in the X-Y plane<br>
where the maximum eigenvalue is found (i.e., ``Z = coords[1]``).

In [ ]:
px.imshow(
    eigen[:, coords[1], :, :],
    facet_col=0,
    labels={'x': 'col', 'y': 'row', 'facet_col': 'rank'},
    title=f'Eigenvalues for plane Z = {coords[1]}.'
)

###################################################################<br>
We are looking at a local property. Let us consider a tiny neighbourhood<br>
around the maximum eigenvalue in the above X-Y plane.

In [ ]:
eigen[0, coords[1], coords[2] - 2:coords[2] + 1, coords[3] - 2:coords[3] + 1]

###################################################################<br>
If we examine the second-largest eigenvalues in this neighbourhood, we can<br>
see that they have the same order of magnitude as the largest ones.

In [ ]:
eigen[1, coords[1], coords[2] - 2:coords[2] + 1, coords[3] - 2:coords[3] + 1]

###################################################################<br>
In contrast, the third-largest eigenvalues are one order of magnitude<br>
smaller.

In [ ]:
eigen[2, coords[1], coords[2] - 2:coords[2] + 1, coords[3] - 2:coords[3] + 1]

###################################################################<br>
Let us visualize the slice of sample data in the X-Y plane where the<br>
maximum eigenvalue is found.

In [ ]:
px.imshow(
    sample[coords[1], :, :],
    zmin=v_min,
    zmax=v_max,
    labels={'x': 'col', 'y': 'row', 'color': 'intensity'},
    title=f'Interactive view of plane Z = {coords[1]}.'
)

###################################################################<br>
Let us visualize the slices of sample data in the X-Z (left) and Y-Z (right)<br>
planes where the maximum eigenvalue is found. The Z axis is the vertical<br>
axis in the subplots below. We can see the expected relative invariance<br>
along the Z axis (corresponding to longitudinal structures in the kidney<br>
tissue), especially in the Y-Z plane (``longitudinal=1``).

In [ ]:
subplots = np.dstack((sample[:, coords[2], :], sample[:, :, coords[3]]))
px.imshow(
    subplots,
    zmin=v_min,
    zmax=v_max,
    facet_col=2,
    labels={'facet_col': 'longitudinal'}
)

###################################################################<br>
As a conclusion, the region about voxel<br>
``(plane, row, column) = coords[1:]`` is<br>
anisotropic in 3D: There is an order of magnitude between the third-largest<br>
eigenvalues on one hand, and the largest and second-largest eigenvalues on<br>
the other hand. We could see this at first glance in figure `Eigenvalues for<br>
plane Z = 1`.

###################################################################<br>
The neighbourhood in question is 'somewhat isotropic' in a plane (which,<br>
here, would be relatively close to the X-Y plane): There is a factor of<br>
less than 2 between the second-largest and largest eigenvalues.<br>
This description is compatible with what we are seeing in the image, i.e., a<br>
stronger gradient across a direction (which, here, would be relatively close<br>
to the row axis) and a weaker gradient perpendicular to it.

###################################################################<br>
In an ellipsoidal representation of the 3D structure tensor [2]_,<br>
we would get the pancake situation.<br>
<br>
.. [2] https://en.wikipedia.org/wiki/Structure_tensor#Interpretation_2